In [2]:
import pickle
import sys
import numpy as np
import pandas as pd
import artm
import seaborn as sns
import matplotlib.pyplot as plt
print artm.version()

from os import path, mkdir
from datetime import datetime
sys.path.insert(0, '..\\modules\\helpers')

import distances_helper as dh 
import print_helper as ph
import create_model_helper as cmh
import build_convex_hull_helper as bchh
import different_models as dm

from plot_helper import PlotMaker
from config_helper import ConfigPaths
from scipy.optimize import minimize

0.8.1


In [3]:
config = ConfigPaths('config_sample_m3.cfg')
print config.models_file_name
models_file = open(config.models_file_name, 'a')

plot_maker = PlotMaker()

batch_vectorizer = artm.BatchVectorizer(data_path=config.output_batches_path,
                                        data_format='batches')
dictionary = artm.Dictionary()
dictionary.load(dictionary_path=config.dictionary_path + '.dict')

Q:\\topic_modeling\\csi_science_collections.git\experiments\pn_model3\np_24_02_kl\models.txt


In [4]:
N_CLOSEST_TOPICS = 15

In [5]:

def create_model_fn_1(n_iteration):
    tmp_model = cmh.create_model(current_dictionary=dictionary, n_topics=20, n_doc_passes=5, seed_value=100 + n_iteration,
                             n_top_tokens=15, p_mass_threshold=0.25)
    tmp_model = cmh.fit_one_model(plot_maker, batch_vectorizer, models_file, config, 
                                  tmp_model, _n_iterations=20, 
                                  _model_name='model_20_m1_iter_{}'.format(n_iteration))
    return tmp_model

def create_model_fn_2(n_iteration):
    tmp_model = cmh.create_model(current_dictionary=dictionary, n_topics=100, n_doc_passes=5, seed_value=100 + n_iteration,
                             n_top_tokens=15, p_mass_threshold=0.25)
    tmp_model = cmh.fit_one_model(plot_maker, batch_vectorizer, models_file, config, 
                                  tmp_model, _n_iterations=20, 
                                  _model_name='model_100_m2_iter_{}'.format(n_iteration))
    return tmp_model

def create_model_fn_3(n_iteration):
    tmp_model = cmh.create_model(current_dictionary=dictionary, n_topics=20, n_doc_passes=5, seed_value=100 + n_iteration,
                            n_top_tokens=15, p_mass_threshold=0.25)
    tmp_model.regularizers.add(artm.DecorrelatorPhiRegularizer(name='decorrelator_phi_regularizer', class_ids=['@default_class']))
    tmp_model.regularizers.add(artm.SmoothSparseThetaRegularizer(name='ss_theta_regularizer'))
    tmp_model.regularizers.add(artm.SmoothSparsePhiRegularizer(name='ss_phi_regularizer', class_ids=['@default_class']))
    tmp_model.regularizers['decorrelator_phi_regularizer'].tau = 300
    tmp_model.regularizers['ss_theta_regularizer'].tau = -5
    tmp_model.regularizers['ss_phi_regularizer'].tau = -20
    tmp_model = cmh.fit_one_model(plot_maker, batch_vectorizer, models_file, config, 
                                  tmp_model, _n_iterations=20, 
                                  _model_name='model_20_m3_iter_{}'.format(n_iteration))
    return tmp_model

def create_model_fn_4(n_iteration):
    tmp_model = cmh.create_model(current_dictionary=dictionary, n_topics=100, n_doc_passes=5, seed_value=100 + n_iteration,
                            n_top_tokens=15, p_mass_threshold=0.25)
    tmp_model.regularizers.add(artm.DecorrelatorPhiRegularizer(name='decorrelator_phi_regularizer', class_ids=['@default_class']))
    tmp_model.regularizers.add(artm.SmoothSparseThetaRegularizer(name='ss_theta_regularizer'))
    tmp_model.regularizers.add(artm.SmoothSparsePhiRegularizer(name='ss_phi_regularizer', class_ids=['@default_class']))
    tmp_model.regularizers['decorrelator_phi_regularizer'].tau = 10
    tmp_model.regularizers['ss_theta_regularizer'].tau = -0.5
    tmp_model.regularizers['ss_phi_regularizer'].tau = -2
    tmp_model = cmh.fit_one_model(plot_maker, batch_vectorizer, models_file, config, 
                                  tmp_model, _n_iterations=20, 
                                  _model_name='model_20_m4_iter_{}'.format(n_iteration))
    return tmp_model

def create_model_fn_20_reg_1(n_iteration):
    tmp_model = create_model(current_dictionary=dictionary, n_topics=20, n_doc_passes=5, seed_value=100 + n_iteration,
                            n_top_tokens=15, p_mass_threshold=0.25)
    tmp_model.regularizers.add(artm.SmoothSparseThetaRegularizer(name='ss_theta_regularizer'))
    tmp_model.regularizers.add(artm.SmoothSparsePhiRegularizer(name='ss_phi_regularizer', class_ids=['@default_class']))
    tmp_model.regularizers['ss_theta_regularizer'].tau = -3
    tmp_model.regularizers['ss_phi_regularizer'].tau = -3
    tmp_model = fit_one_model(tmp_model, _n_iterations=20, _model_name='model_20_reg_1_iter_{}'.format(n_iteration))
    return tmp_model

def create_model_fn_20_reg_2(n_iteration):
    tmp_model = create_model(current_dictionary=dictionary, n_topics=20, n_doc_passes=5, seed_value=100 + n_iteration,
                            n_top_tokens=15, p_mass_threshold=0.25)
    tmp_model.regularizers.add(artm.SmoothSparseThetaRegularizer(name='ss_theta_regularizer'))
    tmp_model.regularizers.add(artm.SmoothSparsePhiRegularizer(name='ss_phi_regularizer', class_ids=['@default_class']))
    tmp_model.regularizers['ss_theta_regularizer'].tau = -5
    tmp_model.regularizers['ss_phi_regularizer'].tau = -5
    tmp_model = fit_one_model(plot_maker, batch_vectorizer, models_file, config, 
                              tmp_model, _n_iterations=20,                               
                              _model_name='model_20_reg_2_iter_{}'.format(n_iteration))
    return tmp_model

def create_model_fn_20_reg_3(n_iteration):
    tmp_model = create_model(current_dictionary=dictionary, n_topics=20, n_doc_passes=5, seed_value=100 + n_iteration,
                            n_top_tokens=15, p_mass_threshold=0.25)
    tmp_model.regularizers.add(artm.SmoothSparseThetaRegularizer(name='ss_theta_regularizer'))
    tmp_model.regularizers.add(artm.SmoothSparsePhiRegularizer(name='ss_phi_regularizer', class_ids=['@default_class']))
    tmp_model.regularizers['ss_theta_regularizer'].tau = -10
    tmp_model.regularizers['ss_phi_regularizer'].tau = -10
    tmp_model = fit_one_model(plot_maker, batch_vectorizer, models_file, config, 
                              tmp_model, _n_iterations=20,             
                              _model_name='model_20_reg_3_iter_{}'.format(n_iteration))
    return tmp_model

def create_model_fn_20_reg_4(n_iteration):
    tmp_model = create_model(current_dictionary=dictionary, n_topics=20, n_doc_passes=5, seed_value=100 + n_iteration,
                            n_top_tokens=15, p_mass_threshold=0.25)
    tmp_model.regularizers.add(artm.DecorrelatorPhiRegularizer(name='decorrelator_phi_regularizer', class_ids=['@default_class']))
    tmp_model.regularizers.add(artm.SmoothSparseThetaRegularizer(name='ss_theta_regularizer'))
    tmp_model.regularizers.add(artm.SmoothSparsePhiRegularizer(name='ss_phi_regularizer', class_ids=['@default_class']))
    tmp_model.regularizers['ss_theta_regularizer'].tau = -5
    tmp_model.regularizers['ss_phi_regularizer'].tau = -5
    tmp_model.regularizers['decorrelator_phi_regularizer'].tau = -10 
    tmp_model = fit_one_model(plot_maker, batch_vectorizer, models_file, config, 
                              tmp_model, _n_iterations=20,             
                              _model_name='model_20_reg_4_iter_{}'.format(n_iteration))
    return tmp_model

def create_model_fn_20_complex_reg_1(n_iteration):
    n_topics = 20
    common_topics = [u'topic_0', u'topic_1']
    subject_topics = list(set([u'topic_{}'.format(idx) for idx in range(2, 20)]) - set(common_topics))
    tmp_model = create_model_complex(current_dictionary=dictionary, n_topics=n_topics, n_doc_passes=5, 
        seed_value=100 + n_iteration, n_top_tokens=15, p_mass_threshold=0.25, 
        common_topics=common_topics, subject_topics=subject_topics)
    # subject topics
    tmp_model.regularizers.add(artm.SmoothSparseThetaRegularizer(name='ss_theta_regularizer_subject', 
        topic_names=subject_topics))
    tmp_model.regularizers.add(artm.SmoothSparsePhiRegularizer(name='ss_phi_regularizer_subject', 
        topic_names=subject_topics, class_ids=['@default_class']))
    tmp_model.regularizers.add(artm.DecorrelatorPhiRegularizer(name='decorrelator_phi_regularizer_subject', 
        topic_names=subject_topics, class_ids=['@default_class']))
    tmp_model.regularizers['ss_theta_regularizer_subject'].tau = -0.5
    tmp_model.regularizers['ss_phi_regularizer_subject'].tau = -0.5
    tmp_model.regularizers['decorrelator_phi_regularizer_subject'].tau = -10

    # common topics
    tmp_model.regularizers.add(artm.SmoothSparseThetaRegularizer(name='ss_theta_regularizer_common', 
        topic_names=subject_topics))
    tmp_model.regularizers.add(artm.SmoothSparsePhiRegularizer(name='ss_phi_regularizer_common', 
        topic_names=subject_topics, class_ids=['@default_class']))
#     tmp_model.regularizers.add(artm.DecorrelatorPhiRegularizer(name='decorrelator_phi_regularizer_common', 
#         topic_names=subject_topics, class_ids=['@default_class']))
    tmp_model.regularizers['ss_theta_regularizer_common'].tau = 0.5
    tmp_model.regularizers['ss_phi_regularizer_common'].tau = 0.5
#     tmp_model.regularizers['decorrelator_phi_regularizer_common'].tau = -10

    tmp_model = fit_one_model_complex(plot_maker, batch_vectorizer, models_file, config, 
                                      tmp_model, _n_iterations=20,             
                                      _model_name='model_20_complex_reg_1_iter_{}'.format(n_iteration))
    return tmp_model

In [6]:
def save_pickle_file(dists, filename, _path=config.experiment_path):
    pickle_filename = path.join(_path, filename)
    pickle_file = open(pickle_filename, 'wb')
    pickle.dump(dists, pickle_file)
    pickle_file.close()
def save_model_pickle(_model_name, _model, _save=True):
    phi = _model.get_phi()
    phi = phi[(phi.T != 0).any()]
    theta = _model.get_theta()    
    saved_top_tokens = _model.score_tracker['top_tokens_score'].last_tokens
    if _save:
        save_pickle_file(phi, 'phi_{}.p'.format(_model_name))
        save_pickle_file(theta, 'theta_{}.p'.format(_model_name))
        save_pickle_file(saved_top_tokens, 'saved_top_tokens_{}.p'.format(_model_name))
    return phi, theta, saved_top_tokens
def load_pickle_file(filename, _path=config.experiment_path):
    pickle_filename = path.join(_path, filename)
    pickle_file = open(pickle_filename, 'rb')
    p_file = pickle.load(pickle_file)
    pickle_file.close()
    return p_file
def load_model_pickle(_model_name, _distance_name=None, _path=config.experiment_path):
    phi = load_pickle_file('phi_{}.p'.format(_model_name), _path)
    theta = load_pickle_file('theta_{}.p'.format(_model_name), _path)
    saved_top_tokens = load_pickle_file('saved_top_tokens_{}.p'.format(_model_name), _path)
    distances = None
    if _distance_name is not None:
        distances = load_pickle_file('{}.p'.format(_distance_name), _path)
    return phi, theta, saved_top_tokens, distances



Загрузим оригинальный sample датасет (от model3), до этого скопировав в папку с batches нужные pickle файлы модели.
Сначала провизуалируем по одной итерации каждой новой модели, а потом будем итерационно строить выпуклую оболочку для каждой модели по отдельности и затем сравнивать их. 

In [7]:
phi_original, theta_original, saved_top_tokens_original, distances_hellinger_model_original = load_model_pickle('model3', 'distances_hellinger_model3', config.output_batches_path)
print phi_original.shape, theta_original.shape

(2216, 100) (100, 3446)


Запустить несколько раз с разным рандомом. Следить за тем, чтобы накапливались только независимые темы. Каждый раз смотреть. как проектируется на оригинальную матрицу.

In [8]:
# now some info from run
# distibution of topics in convex hull by iteration
def plot_convex_hull_topics_iterations_distribution(_phi_convex_hull):
    get_iteration_number_fn = lambda x: int(x[x.find('_', 6) + 1 : ])
    phi_convex_hull_iteration_number = [get_iteration_number_fn(col) for col in _phi_convex_hull.columns]
    phi_convex_hull_iteration_number = [(val, phi_convex_hull_iteration_number.count(val), 1.0 * phi_convex_hull_iteration_number.count(val) / len(phi_convex_hull_iteration_number)) for val in set(phi_convex_hull_iteration_number)]
    print(phi_convex_hull_iteration_number)
    
    fig, (ax1, ax2) = plt.subplots(ncols=2, sharey=False, figsize=(15,5))
    sns.barplot([x[0] for x in phi_convex_hull_iteration_number], [x[1] for x in phi_convex_hull_iteration_number], ax=ax1)
    ax1.set_title('Number of topics from each iteration')
    ax1.set_xlabel('n iteration')

    sns.barplot([x[0] for x in phi_convex_hull_iteration_number], [x[2] for x in phi_convex_hull_iteration_number], ax=ax2)
    ax2.set_title('Number of topics from each iteration (%)')
    ax2.set_xlabel('n iteration')
def plot_convex_hull_columns_change(iterations_info):
    fig, (ax1, ax2) = plt.subplots(ncols=2, sharey=False, figsize=(15,5))
    ax1.plot([sum([val['n_topics_to_remove']for val in it['iterations_info_filter']]) for it in iterations_info], 
             color='r', label = 'total')
    ax1.set_title('Num columns to remove')
    get_topic_iteration_fn = lambda x: int(x[x.rfind('_') + 1 :])
    get_topic_filter_iteration_list_fn = lambda x, y: [get_topic_iteration_fn(topic) for topic in x].count(y)
    n_topics_removed_from_current_iteration = [sum([get_topic_filter_iteration_list_fn(val['removed_topics'], indx) for val in it['iterations_info_filter']]) for indx, it in enumerate(iterations_info)]
    ax1.plot(n_topics_removed_from_current_iteration, color='b', label='current iteration')
    ax1.set_xlabel('n iteration')
    ax1.legend()

    ax2.plot([val['phi_convex_hull_shape'][1] for val in iterations_info], color='r')
    ax2.set_title('Num columns of convex hull')
    ax2.set_xlabel('n iteration')
    ax2.legend()
def plot_opt_res_fun(iterations_filtering_info_name):
    %matplotlib inline
    iterations_filtering_info = load_pickle_file(iterations_filtering_info_name)
    fig, (ax1, ax2) = plt.subplots(ncols=2, sharey=False, figsize=(15,5))
    values = [x.fun for item in iterations_filtering_info for val in item for x in val['opt_res'].values()]
    sns.distplot(values, color='r', bins=10, ax=ax1)
    values = [[x.fun for val in item for x in val['opt_res'].values()] for item in iterations_filtering_info]
    for val in values:
        sns.distplot(val, bins=10, ax=ax2)
def plot_opt_res_fun_filtering(iterations_filtering_info_name):
    iterations_info = load_pickle_file(iterations_filtering_info_name)
    fig, (ax1, ax2) = plt.subplots(ncols=2, sharey=False, figsize=(15,5))
    values = [x.fun for item in iterations_info for val in item['iterations_info_filter'] for x in val['opt_res'].values()]
    sns.distplot(values, color='r', bins=10, ax=ax1)
    values = [[x.fun for val in item['iterations_info_filter'] for x in val['opt_res'].values()] for item in iterations_info]
    for val in values:
        sns.distplot(val, bins=10, ax=ax2)

In [8]:
OPT_FUN_THRESHOLD = 0.6
phi_convex_hull_df_m2__0_6, iterations_info_df_m2__0_6, iterations_filtering_info_df_m2__0_6 = bchh \
                                                    .build_convex_hull_delayed_filtering(
                                                     create_model_fn_2,
                                                     bchh.get_topics_to_remove_by_opt_fun_and_distance, 
                                                     phi_original.index,
                                                     init_convex_hull=[], start_iteration=0,
                                                     n_closest_topics_count=2,
                                                     opt_fun_threshold=0.6,
                                                     max_iteration=1, filtering_iteration=1)
plot_convex_hull_topics_iterations_distribution(phi_convex_hull_df_m2__0_6)
# plot_opt_res_fun('iterations_filtering_info_df_m2__0_6')

[2017-02-24 12:44:07.177000] ********** iteration = 1 / 1


NameError: global name 'dictionary' is not defined

In [12]:
def get_opt_x_close_to_th_count(opt_x, global_th=0.95):
    cur_sum, sum_count = 0, 0
    for val in sorted(opt_x)[::-1]:
        if cur_sum + val <= global_th:
            cur_sum += val
            sum_count += 1
        else:
            break
    return sum_count + 1, cur_sum
def get_opt_x_granularity(opt):
    return np.mean([get_opt_x_close_to_th_count(opt_res.x)[0] for topic, opt_res in opt.iteritems()])

# tests

In [13]:
# permutations
def test_permutation_optimization(_phi_original, dist_fn, n_closest_topics_count, _debug_print=False):
    _phi_original = phi_original
    n_topics = _phi_original.shape[1]
    indices = np.random.choice(n_topics, n_topics, replace=False)
    phi_permuted = _phi_original.iloc[:, indices]
    distances = bchh.calculate_distances(dist_fn, phi_permuted, _phi_original)
    opt_res_convex_hull = bchh.get_optimization_result(dh.hellinger_dist, None, phi_permuted, _phi_original,
                                                             distances, n_closest_topics_count)
    print get_opt_x_granularity(opt_res_convex_hull)
    
    if _debug_print:
        for _, item in opt_res_convex_hull.items():
             print item.optimized_column, item.fun, item.x[0:3], item.column_names[0:3]
    matched_columns_count = 0
    good_x_coefficient_count = 0
    for _, item in opt_res_convex_hull.items():
        matched_columns_count += item.optimized_column == item.column_names[0]
        good_x_coefficient_count += item.x[0] > 0.99
    matched_columns = 100.0 * matched_columns_count / n_topics
    good_x_coefficient = 100.0 * good_x_coefficient_count / n_topics
    print 'matched_columns = {}%, good_x_coefficient = {}%'.format(matched_columns, good_x_coefficient)
    return matched_columns, good_x_coefficient

In [14]:
_,_ = test_permutation_optimization(phi_original, dh.hellinger_dist, n_closest_topics_count=5)

1.0
matched_columns = 100.0%, good_x_coefficient = 100.0%


In [26]:
def test_combination_optimization(_phi_original, dist_fn, n_closest_topics_count, n_runs, _debug_print=False):
    _eps = 0.15
    not_found_count_list, matched_coeffs_count_list = [], []
    for n_iter in range(n_runs):               
        word_count, topics_count = phi_original.shape
        combination_count = np.random.randint(2, 7)
        combination_indices = np.random.choice(topics_count, combination_count, replace=False)
        phi_combination = phi_original.iloc[:, combination_indices]
        single_combination_weights = np.random.uniform(0, 1, combination_count)
        single_combination_weights = single_combination_weights / sum(single_combination_weights)
        combination_weights = np.array([single_combination_weights]*word_count)
        combination_weights = pd.DataFrame(data=combination_weights, index=phi_combination.index, columns=phi_combination.columns)
        phi_combination = phi_combination.multiply(combination_weights)
        phi_combination = pd.DataFrame(phi_combination.sum(axis=1), index=phi_combination.index, columns=['combination_0'])

        distances = bchh.calculate_distances(dh.hellinger_dist, phi_combination, _phi_original)
        opt_res_convex_hull = bchh.get_optimization_result(dh.hellinger_dist, None, phi_combination, _phi_original,
                                                                 distances, n_closest_topics_count)
        results = []
        for idx, col_name in enumerate(phi_original.columns[combination_indices]):
            opt_cols_names = list(opt_res_convex_hull['combination_0'].column_names)
            col_name_opt_idx = opt_cols_names.index(col_name) if col_name in opt_cols_names else -1
            col_name_opt_x = opt_res_convex_hull['combination_0'].x[col_name_opt_idx] if col_name_opt_idx != -1 else 0
            col_name_opt_idx = col_name_opt_idx if col_name_opt_idx != -1 else \
                               (0 if single_combination_weights[idx] < 0.1 else -1)
            results.append((col_name, single_combination_weights[idx], col_name_opt_x))           
        not_found_count = len([res for res in results if res[2] == -1]) * 100.0 / combination_count
        matched_coeffs_count = 100.0 * len([res for res in results if abs(res[1] - res[2]) < _eps]) / combination_count
        if _debug_print or not_found_count != 0 or matched_coeffs_count != 100: 
            print('it = {} / {} not_found_columns_count = {}%, matched_coeffs_count = {}%'.format(n_iter + 1, n_runs, 
                                                                            not_found_count, matched_coeffs_count))
            for r in results:
                print r
        not_found_count_list.append(not_found_count)
        matched_coeffs_count_list.append(matched_coeffs_count)
    total_not_found_count = sum(not_found_count_list) / n_runs
    total_matched_coeffs_count = sum(matched_coeffs_count_list) / n_runs
    print('total not_found_columns_count = {}%, matched_coeffs_count = {}%'.format(total_not_found_count,
                                                                                   total_matched_coeffs_count))
    return total_not_found_count, total_matched_coeffs_count

In [27]:
_,_ = test_combination_optimization(phi_original, dh.hellinger_dist, n_closest_topics_count=5, n_runs=100, _debug_print=False)

total not_found_columns_count = 0.0%, matched_coeffs_count = 100.0%
